# Load libraries / create tokenizer object 

In [59]:
import pandas as pd 
import string 
import numpy as np 
from nltk import word_tokenize
from nltk.corpus import stopwords   
from nltk.stem import PorterStemmer
from collections import Counter 
import re 

# Preprocess data 

In [57]:
ps = PorterStemmer()
stop = set(stopwords.words('english'))

def read_txt(file_path): 
    # read file contents by line 
    file = open(file_path, "r", encoding="utf-8")
    text = file.read() 
    text = re.sub(r'\n', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    text = text.lower()
    tokenized_words = word_tokenize(text)
    tokenized_words = [w for w in tokenized_words if w not in stop]
    tokenized_words = [ps.stem(word) for word in tokenized_words]
    return tokenized_words

websites = [] 
websites = websites.append(read_txt("movie.txt"))
print(websites)

  0     1    2    3     4       5    6      7     8    9     ...    1162 1163  \
0  one  movi   ai  top  head  irobot  one  smith  sinc  ive  ...  season    6   

   1164     1165    1166    1167 1168      1169    1170 1171  
0  main  product  center  around   ai  blackbox  neural  net  

[1 rows x 1172 columns]


# TF / IDF Frequency

In [60]:
doc_occurrences = {}
def get_frequencies(website): 
    word_dict = {} # stores the frequency of each word in a doc 
    for word in website: 
        if word in word_dict: 
            word_dict[word] += 1 
        else: 
            word_dict[word] = 1 
    return word_dict 

def get_doc_occurrences(website_dict, websites, doc_occurrences): # get the number of websites a word appears in 
    for key in website_dict: 
        if key not in doc_occurrences: 
            ct = 0 
            for website in websites: 
                if key in website: 
                    ct = ct + 1 
            doc_occurrences[key] = ct 


def calculate_tf_idf(doc_occurrences, word_dict, )






AttributeError: 'int' object has no attribute 'split'